<a href="https://colab.research.google.com/github/huhlim/cg2all/blob/main/cryo_em_minimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This notebook performs local optimization of a protein model structure against a cryo-EM density map using __cryo_em_minimizer.py__ This task is running very slow on the Google Colab __CPU__ runtime. Thus, it is highly recommended to use a __GPU__ runtime with enough VRAM. Because it only performs local optimization, the input structure should be roughly fitted to the input electron density map.

---

In [ ]:
#@title Install cg2all package (takes 4-5 minutes)
%%bash

gpu_available=$(nvidia-smi | grep "CUDA Version" | wc -l)
if [[ $gpu_available == 1 ]]; then
    echo "This notebook is running on a GPU runtime."
    pip install dgl -f https://data.dgl.ai/wheels/cu116/repo.html &> /dev/null
else
    echo "This notebook is running on a CPU runtime."
fi

pip install -q git+http://github.com/huhlim/cg2all@v1.5 &> /dev/null
pip install -q py3Dmol gdown mrcfile &> /dev/null

In [ ]:
#@title Download a model checkpoint file (optional, takes a few seconds)
#@markdown This step downloads a necessary PyTorch model checkpoint file. If you did not run this step and the file is missing, then the script will download it automatically later.

import cg2all.lib.libmodel
from cg2all.lib.libconfig import MODEL_HOME

for model_type in ["CalphaBasedModel", "ResidueBasedModel"]:
    ckpt_fn = MODEL_HOME / f"{model_type}.ckpt"
    if not ckpt_fn.exists():
        cg2all.lib.libmodel.download_ckpt_file(model_type, ckpt_fn)


In [ ]:
#@title Set up py3Dmol for structure display

import mdtraj
import py3Dmol

def display(pdb_fn, representation="cartoon", is_traj=False):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if is_traj:
        view.addModelsAsFrames(open(pdb_fn,'r').read(),'pdb')
    else:
        view.addModel(open(pdb_fn,'r').read(),'pdb')

    if representation == "cartoon":
        view.setStyle({'cartoon': {'color':'spectrum'}})
        #
        view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom': ["N", "C", "O"],'invert':True}]},
                        {'stick':{'colorscheme':"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':"WhiteCarbon",'radius':0.3}})

    elif representation == "ball+stick":
        view.setStyle({"sphere": {"color": "spectrum", "radius": 1.0}})

    else:
        raise NotImplementedError(representation)

    view.zoomTo()
    if is_traj:
        view.animate({'loop': 'forward', "interval": 500})

    return view

---

In [ ]:
#@title Run local optimization

# upload a PDB file
import requests
from google.colab import files

coarse_grained_model_type = "ResidueBasedModel" #@param ["CalphaBasedModel", "ResidueBasedModel"]
segmentation_method = "chain" #@param ["None", "chain", "segment"]
if segmentation_method == "None":
    segmentation_method = None
use_example = True #@param {type:"boolean"}
#@markdown - An example input PDB and electron density map files will be downloaded from [our repository](https://github.com/huhlim/cg2all/tree/main/tests).

if use_example:
    url = "https://raw.githubusercontent.com/huhlim/cg2all/main/tests/3isr.af2.pdb"
    input_pdb = url.split("/")[-1]
    with open(input_pdb, "wt") as fout:
        fout.write(requests.get(url).text)
    #
    url = "https://raw.githubusercontent.com/huhlim/cg2all/main/tests/3isr_5.mrc"
    input_map = url.split("/")[-1]
    with open(input_map, "wb") as fout:
        fout.write(requests.get(url).content)

else:
    input_pdb = files.upload()
    input_pdb = list(input_pdb)[0]
    input_map = files.upload()
    input_map = list(input_map)[0]

number_of_snapshots = 10 #@param {type: "slider", min:1, max:100}
#@markdown - The number of minimization steps is 10 x number_of_snapshots.
n_step = number_of_snapshots * 10

import pathlib
output_prefix = pathlib.Path(input_map).stem + "+" + pathlib.Path(input_pdb).stem

!cryo_em_minimizer --pdb $input_pdb --map $input_map --output $output_prefix --cg $coarse_grained_model_type --all -n $n_step --output_freq 10 --segment $segmentation_method --uniform_restraint

pdb_fn_s = list(pathlib.Path(output_prefix).glob("min.*.pdb"))
pdb_fn_s.sort(key=lambda fn: int(fn.name.split(".")[-2]))
if len(pdb_fn_s) > 10:
    pdb_fn_s = pdb_fn_s[::len(pdb_fn_s)//10]
    print(f"Displaying only {len(pdb_fn_s)}")

traj = mdtraj.load(pdb_fn_s)
traj.save("display.pdb")
display("display.pdb", representation="cartoon", is_traj=True).show()

In [ ]:
#@title Download the optimized files

import os
zip = f"{output_prefix}.zip"
os.system(f"zip -r {zip} {output_prefix}")
files.download(zip)